In [1]:
### import collections
import json
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from mab import algs


class ONN(nn.Module):
    def __init__(self, features_size, max_num_hidden_layers, qtd_neuron_per_hidden_layer, n_classes, batch_size=1,
                 b=0.99, n=0.01, s=0.2, use_cuda=False):
        super(ONN, self).__init__()

        if torch.cuda.is_available() and use_cuda:
            print("Using CUDA :]")

        self.device = torch.device(
            "cuda:0" if torch.cuda.is_available() and use_cuda else "cpu")

        self.features_size = features_size
        self.max_num_hidden_layers = max_num_hidden_layers
        self.qtd_neuron_per_hidden_layer = qtd_neuron_per_hidden_layer
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.b = Parameter(torch.tensor(
            b), requires_grad=False).to(self.device)
        self.n = Parameter(torch.tensor(
            n), requires_grad=False).to(self.device)
        self.s = Parameter(torch.tensor(
            s), requires_grad=False).to(self.device)

        self.hidden_layers = []
        self.output_layers = []

        self.hidden_layers.append(
            nn.Linear(features_size, qtd_neuron_per_hidden_layer))

        for i in range(max_num_hidden_layers - 1):
            self.hidden_layers.append(
                nn.Linear(qtd_neuron_per_hidden_layer, qtd_neuron_per_hidden_layer))

        for i in range(max_num_hidden_layers):
            self.output_layers.append(
                nn.Linear(qtd_neuron_per_hidden_layer, n_classes))

        self.hidden_layers = nn.ModuleList(self.hidden_layers).to(self.device)
        self.output_layers = nn.ModuleList(self.output_layers).to(self.device)

        self.alpha = Parameter(torch.Tensor(self.max_num_hidden_layers).fill_(1 / (self.max_num_hidden_layers + 1)),
                               requires_grad=False).to(
            self.device)

        self.loss_array = []
        self.tp=0.1
        self.tn=0.1
        self.fp=0.1
        self.fn=0.1
        self.bal_acc = 0
        self.bal_acc_list = []
        self.sen = 0
        self.spec = 0
        
        self.protected_pos = 0.1
        self.protected_neg = 0.1
        self.non_protected_pos = 0.1
        self.non_protected_neg = 0.1
        self.stp_score=0
        
        self.tp_protected = 0.1
        self.fn_protected = 0.1
        self.tp_non_protected = 0.1
        self.fn_non_protected = 0.1
        self.eqop_score=0

    def zero_grad(self):
        for i in range(self.max_num_hidden_layers):
            self.output_layers[i].weight.grad.data.fill_(0)
            self.output_layers[i].bias.grad.data.fill_(0)
            self.hidden_layers[i].weight.grad.data.fill_(0)
            self.hidden_layers[i].bias.grad.data.fill_(0)

    def update_weights(self, X, Y, weight, show_loss, test):
        Y = torch.from_numpy(Y).to(self.device)
        if test=='yes':
            prediction_1 = self.predict_1(X)
            self.update_eval_metrics(prediction_1,Y)
            self.update_stp_score(prediction_1,X)
            self.update_eqop_score(prediction_1,X,Y)
        
        predictions_per_layer = self.forward(X)
        losses_per_layer = []

        for out in predictions_per_layer:
            criterion = nn.CrossEntropyLoss().to(self.device)
            loss = criterion(out.view(self.batch_size, self.n_classes), Y.view(
                self.batch_size).long())
            losses_per_layer.append(loss*weight)

        w = [None] * len(losses_per_layer)
        b = [None] * len(losses_per_layer)
        
        with torch.no_grad():     #Context-manager that disabled gradient calculation
            self.weights_output_layers = list()
            self.biases_output_layers = list()
            for i in range(len(losses_per_layer)):
                losses_per_layer[i].backward(retain_graph=True)
                self.output_layers[i].weight.data -= self.n * \
                                                   self.alpha[i] * self.output_layers[i].weight.grad.data
                self.output_layers[i].bias.data -= self.n * \
                                                 self.alpha[i] * self.output_layers[i].bias.grad.data
                self.weights_output_layers.append(self.output_layers[i].weight.data) 
                self.biases_output_layers.append(self.output_layers[i].bias.data)
                
                for j in range(i + 1):
                    if w[j] is None:
                        w[j] = self.alpha[i] * self.hidden_layers[j].weight.grad.data
                        b[j] = self.alpha[i] * self.hidden_layers[j].bias.grad.data
                    else:
                        w[j] += self.alpha[i] * self.hidden_layers[j].weight.grad.data
                        b[j] += self.alpha[i] * self.hidden_layers[j].bias.grad.data
                self.zero_grad()
            self.weights_hidden_layers= list()
            self.biases_hidden_layers = list()
            
            for i in range(len(losses_per_layer)):
                self.hidden_layers[i].weight.data -= self.n * w[i]
                self.hidden_layers[i].bias.data -= self.n * b[i]
                self.weights_hidden_layers.append(self.hidden_layers[i].weight.data)
                self.biases_hidden_layers.append(self.hidden_layers[i].bias.data)
            for i in range(len(losses_per_layer)):
                self.alpha[i] *= torch.pow(self.b, losses_per_layer[i])
                self.alpha[i] = torch.max(
                  self.alpha[i], self.s / self.max_num_hidden_layers)

        z_t = torch.sum(self.alpha)

        self.alpha = Parameter(
            self.alpha / z_t, requires_grad=False).to(self.device)
        if show_loss:
            
            real_output = torch.sum(torch.mul(
                self.alpha.view(self.max_num_hidden_layers, 1).repeat(1, self.batch_size).view(
                    self.max_num_hidden_layers, self.batch_size, 1), predictions_per_layer), 0)
            criterion = nn.CrossEntropyLoss().to(self.device)
            loss = criterion(real_output.view(self.batch_size, self.n_classes), Y.view(self.batch_size).long())
            self.loss_array.append(loss)
            if (len(self.loss_array) % 1000) == 0:
                #print("WARNING: Set 'show_loss' to 'False' when not debugging. "
                #      "It will deteriorate the fitting performance.")
                loss = torch.Tensor(self.loss_array).mean().cpu().numpy()
                #print("Alpha:" + str(self.alpha.data.cpu().numpy()))
                print("Training Loss: " + str(loss))
                print("Balanced accuracy: " + str(self.bal_acc))
                print("Sensitivity: " + str(self.sen))
                print("Specificity: " + str(self.spec))
                print("Stp score: " + str(self.stp_score))
                print("Eqop score: " + str(self.eqop_score))
                self.loss_array.clear()

    def get_weights(self,network_name):
        
        if network_name == 'global':
            weights_output_layers = list()
            biases_output_layers = list()
            weights_hidden_layers = list()
            biases_hidden_layers= list()
            for i in range(self.max_num_hidden_layers):
                weights_output_layers.append(self.output_layers[i].weight.data) 
                biases_output_layers.append(self.output_layers[i].bias.data)
                weights_hidden_layers.append(self.hidden_layers[i].weight.data)
                biases_hidden_layers.append(self.hidden_layers[i].bias.data)
               
            return self.alpha, weights_output_layers, biases_output_layers, weights_hidden_layers, biases_hidden_layers
        else:
            return self.alpha, self.weights_output_layers, self.biases_output_layers, self.weights_hidden_layers, self.biases_hidden_layers
    
    def set_weights(self, alpha, w_output_layer, b_output_layer, w_hidden_layer, b_hidden_layer):
        
        for i in range(self.max_num_hidden_layers):
            self.alpha[i] = alpha[i].clone().detach()
            self.output_layers[i].weight.data = w_output_layer[i].clone().detach()
            self.output_layers[i].bias.data = b_output_layer [i].clone().detach()
            self.hidden_layers[i].weight.data =  w_hidden_layer [i].clone().detach()
            self.hidden_layers[i].bias.data = b_hidden_layer [i].clone().detach()    
    def forward(self, X):
        hidden_connections = []

        X = torch.from_numpy(X).float().to(self.device)

        x = F.relu(self.hidden_layers[0](X))
        hidden_connections.append(x)

        for i in range(1, self.max_num_hidden_layers):
            hidden_connections.append(
                F.relu(self.hidden_layers[i](hidden_connections[i - 1])))

        output_class = []

        for i in range(self.max_num_hidden_layers):
            output_class.append(self.output_layers[i](hidden_connections[i]))

        pred_per_layer = torch.stack(output_class)

        return pred_per_layer

    def validate_input_X(self, data):
        
        if len(data.shape) != 2:
            raise Exception(
                "Wrong dimension for this X data. It should have only two dimensions.")

    def validate_input_Y(self, data):
        if len(data.shape) != 1:
            raise Exception(
                "Wrong dimension for this Y data. It should have only one dimensions.")

    def partial_fit_(self, X_data, Y_data, weight, show_loss=True, test='yes'):
        
        self.validate_input_X(X_data)
        self.validate_input_Y(Y_data)
        self.update_weights(X_data, Y_data, weight, show_loss, test)

    def partial_fit(self, X_data, Y_data, weight, show_loss=True, test='yes'):
        
        self.partial_fit_(X_data, Y_data, weight, show_loss, test)

    def reset_eval_metrics(self):
        self.tp=0.1
        self.tn=0.1
        self.fp=0.1
        self.fn=0.1
        self.bal_acc = 0
        self.bal_acc_list = []
        self.sen = 0
        self.spec = 0
        
        self.protected_pos = 0.1
        self.protected_neg = 0.1
        self.non_protected_pos = 0.1
        self.non_protected_neg = 0.1
        self.stp_score=0
        
        self.tp_protected = 0.1
        self.fn_protected = 0.1
        self.tp_non_protected = 0.1
        self.fn_non_protected = 0.1
        self.eqop_score=0
    def update_eval_metrics(self,prediction_1,Y):
        if prediction_1==1 and Y==1:
            self.tp+=1
        elif prediction_1==1 and Y==0:
            self.fp+=1
        elif prediction_1==0 and Y==1:
            self.fn+=1
        else:
            self.tn+=1
        
        self.sen = self.tp/(self.tp + self.fn)
        self.spec= self.tn/(self.tn + self.fp)
        self.bal_acc = (self.sen + self.spec)/2
        self.bal_acc_list.append(self.bal_acc)
            
    def update_stp_score(self,prediction_1,X):
        if X[0][sa_index] == p_Group:
            if prediction_1 == 1:
                self.protected_pos += 1.
            else:
                self.protected_neg += 1.
        else:
            if prediction_1 == 1:
                self.non_protected_pos += 1.
            else:
                self.non_protected_neg += 1.
            
        C_prot = (self.protected_pos) / (self.protected_pos + self.protected_neg)
        C_non_prot = (self.non_protected_pos) / (self.non_protected_pos + self.non_protected_neg)

        self.stp_score = C_non_prot - C_prot
    
    def update_eqop_score(self,prediction_1,X,Y):
        if X[0][sa_index] == p_Group:
            if prediction_1==1 and Y==1:
                self.tp_protected += 1.
            elif prediction_1==0 and Y==1:
                self.fn_protected += 1.
        else:
            if prediction_1==1 and Y==1:
                self.tp_non_protected += 1.
            elif prediction_1==0 and Y==1:
                self.fn_non_protected += 1.
            
        tpr_protected = self.tp_protected / (self.tp_protected + self.fn_protected)
        tpr_non_protected = self.tp_non_protected / (self.tp_non_protected + self.fn_non_protected)
        self.eqop_score = tpr_non_protected - tpr_protected
    
    def predict_1(self, X_data):
        self.validate_input_X(X_data)
        return torch.argmax(torch.sum(torch.mul(
            self.alpha.view(self.max_num_hidden_layers, 1).repeat(1, 1).view(
                self.max_num_hidden_layers, len(X_data), 1), self.forward(X_data)), 0), dim=1).cpu().numpy()
    def predict_(self, X_data):
        self.validate_input_X(X_data)
        return torch.argmax(torch.sum(torch.mul(
            self.alpha.view(self.max_num_hidden_layers, 1).repeat(1, len(X_data)).view(
                self.max_num_hidden_layers, len(X_data), 1), self.forward(X_data)), 0), dim=1).cpu().numpy()

    def predict(self, X_data):
        pred = self.predict_(X_data)
        return pred

    def export_params_to_json(self):
        state_dict = self.state_dict()
        params_gp = {}
        for key, tensor in state_dict.items():
            params_gp[key] = tensor.cpu().numpy().tolist()

        return json.dumps(params_gp)

    def load_params_from_json(self, json_data):
        params = json.loads(json_data)
        o_dict = collections.OrderedDict()
        for key, tensor in params.items():
            o_dict[key] = torch.tensor(tensor).to(self.device)
        self.load_state_dict(o_dict)


class ONN_THS(ONN):
    def __init__(self, features_size, max_num_hidden_layers, qtd_neuron_per_hidden_layer, n_classes, b=0.99, n=0.01,
                 s=0.2, e=[0.5, 0.35, 0.2, 0.1, 0.05], use_cuda=False):
        super().__init__(features_size, max_num_hidden_layers, qtd_neuron_per_hidden_layer, n_classes, b=b, n=n, s=s,
                         use_cuda=use_cuda)
        self.e = Parameter(torch.tensor(e), requires_grad=False)
        self.arms_values = Parameter(
            torch.arange(n_classes), requires_grad=False)
        self.explorations_mab = []

        for i in range(n_classes):
            self.explorations_mab.append(algs.ThompsomSampling(len(e)))

    def partial_fit(self, X_data, Y_data, exp_factor, show_loss=True):
        self.partial_fit_(X_data, Y_data, show_loss)
        self.explorations_mab[Y_data[0]].reward(exp_factor)

    def predict(self, X_data):
        pred = self.predict_(X_data)[0]
        exp_factor = self.explorations_mab[pred].select()[0]
        if np.random.uniform() < self.e[exp_factor]:
            removed_arms = self.arms_values.clone().numpy().tolist()
            removed_arms.remove(pred)
            return random.choice(removed_arms), exp_factor

        return pred, exp_factor


In [2]:
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from imblearn.datasets import make_imbalance
import numpy as np

In [3]:
# import urllib2
import os, sys
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn import feature_extraction
from sklearn import preprocessing
from random import seed, shuffle

# import utils as ut

SEED = 1234
seed(SEED)
np.random.seed(SEED)


def load_default():
    FEATURES_CLASSIFICATION = ["LIMIT_BAL","SEX","EDUCATION","MARRIAGE","AGE","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]  # features to be used for classification
    CONT_VARIABLES = ["AGE","BILL_AMT1","BILL_AMT2","BILL_AMT3","BILL_AMT4","BILL_AMT5","BILL_AMT6","PAY_AMT1","PAY_AMT2","PAY_AMT3","PAY_AMT4","PAY_AMT5","PAY_AMT6"]  # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "y"  # the decision variable
    SENSITIVE_ATTRS = ["SEX"]
    CAT_VARIABLES = ["LIMIT_BAL","SEX","EDUCATION","MARRIAGE","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6"]
    CAT_VARIABLES_INDICES = [0,2,3,5,6,7,8,9,10]
    
    INPUT_FILE = "./datasets/default.csv"

    df = pd.read_csv(INPUT_FILE)
    
    # convert to np array
    data = df.to_dict('list')
    
    for k in data.keys():
        data[k] = np.array(data[k])

    """ Feature normalization and one hot encoding """

    # convert class label 0 to -1
    y = data[CLASS_FEATURE]
    #y[y == "yes"] = 1
    #y[y == 'no'] = -1
    y = np.array([int(k) for k in y])

    X = np.array([]).reshape(len(y), 0)  # empty array with num rows same as num examples, will hstack the features to it
    
    x_control = defaultdict(list)
    i=0
    feature_names = []
    
    for attr in FEATURES_CLASSIFICATION:
        vals = data[attr]
        
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals)  # 0 mean and 1 variance
            vals = np.reshape(vals, (len(y), -1))  # convert from 1-d arr to a 2-d arr with one col
            

        else:  # for binary categorical variables, the label binarizer uses just one var instead of two
            lb = preprocessing.LabelBinarizer()
            lb.fit(vals)
            vals = lb.transform(vals)
            '''
            if attr == 'SEX':
                print(lb.classes_)
                print(lb.transform(lb.classes_))
            '''
            #if attr == 'job':
            #    print(lb.classes_)
            #    print(lb.transform(lb.classes_))
            
           
            
            
        # add to sensitive features dict
        if attr in SENSITIVE_ATTRS:
            x_control[attr] = vals
            

        # add to learnable features
        X = np.hstack((X, vals))
        
        if attr in CONT_VARIABLES:  # continuous feature, just append the name
            feature_names.append(attr)
        else:  # categorical features
            if vals.shape[1] == 1:  # binary features that passed through lib binarizer
                feature_names.append(attr)
            else:
                for k in lb.classes_:  # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    # convert the sensitive feature to 1-d array
    
    x_control = dict(x_control)
    
    for k in x_control.keys():
        assert (x_control[k].shape[1] == 1)  # make sure that the sensitive feature is binary after one hot encoding
        x_control[k] = np.array(x_control[k]).flatten()
    
    feature_names.append('target')
    # '0' is 'female'
   
    
    return X, y, feature_names.index(SENSITIVE_ATTRS[0]), 0, x_control

X,y, sa_index, p_Group, x_control= load_default()

np_Group = 1 #non-protected group's sa_value



In [4]:
COMPAS_INPUT_FILE = "./datasets/default.csv"

df = pd.read_csv(COMPAS_INPUT_FILE)
age_group_1 = []
age_group_2 = []
age_group_3 = []
for i in range(len(df)):
    if df['AGE'].iloc[i]>0 and df['AGE'].iloc[i]<30:
        age_group_1.append(i)
    elif df['AGE'].iloc[i]>29 and df['AGE'].iloc[i]<40:
        age_group_2.append(i)
    elif df['AGE'].iloc[i]>39:
        age_group_3.append(i)
   

In [5]:
Xtr1 = np.empty((0,0))
Ytr1 = np.empty(0)
for i in age_group_1:
    if np.size(Xtr1)==0:
        print("bismillah")
        Xtr1 = X[i]
        Ytr1 = y[i]
    else:
        Xtr1 = np.vstack((Xtr1,X[i]))
        Ytr1 = np.append(Ytr1,y[i])

Xtr2 = np.empty((0,0))
Ytr2 = np.empty(0)
for i in age_group_2:
    if np.size(Xtr2)==0:
        print("bismillah")
        Xtr2 = X[i]
        Ytr2 = y[i]
    else:
        Xtr2 = np.vstack((Xtr2,X[i]))
        Ytr2 = np.append(Ytr2,y[i])
        
Xtr3 = np.empty((0,0))
Ytr3 = np.empty(0)
for i in age_group_3:
    if np.size(Xtr3)==0:
        print("bismillah")
        Xtr3 = X[i]
        Ytr3 = y[i]
    else:
        Xtr3 = np.vstack((Xtr3,X[i]))
        Ytr3 = np.append(Ytr3,y[i])
print(len(Xtr3))
print(len(Ytr3))
print(Ytr3)

bismillah
bismillah
bismillah
9144
9144
[0 0 0 ... 0 1 1]


In [6]:
labels = Ytr3
unique, counts = np.unique(labels, return_counts=True)
count_ap_dict = dict(zip(unique, counts))
print(count_ap_dict)
minority_label=1
majority_label = 0        

{0: 6981, 1: 2163}


In [7]:
clients = {}
client_data_testx = []
client_data_testy = []
x_train, x_test, y_train, y_test = train_test_split(Xtr1,Ytr1,test_size=0.2)
Xtr1 = x_train
Xte1 = x_test
Ytr1 = y_train
Yte1 = y_test
Xtr = x_train
client_data_testx.append(Xte1)
client_data_testy.append(Yte1)
####
x_train, x_test, y_train, y_test = train_test_split(Xtr2,Ytr2,test_size=0.2)
Xtr2 = x_train
Xte2 = x_test
Ytr2 = y_train
Yte2 = y_test
client_data_testx.append(Xte2)
client_data_testy.append(Yte2)
####
x_train, x_test, y_train, y_test = train_test_split(Xtr3,Ytr3,test_size=0.2)
Xtr3 = x_train
Xte3 = x_test
Ytr3 = y_train
Yte3 = y_test
client_data_testx.append(Xte3)
client_data_testy.append(Yte3)
####

#client_data_testx.append(Xte4)
#client_data_testy.append(Yte4)


In [8]:
#concatnate teset data
x_test_new = np.concatenate((client_data_testx[0], client_data_testx[1]), axis=0)
x_test_new = np.concatenate((x_test_new, client_data_testx[2]), axis=0)
y_test_new = np.concatenate((client_data_testy[0], client_data_testy[1]), axis=0)
y_test_new = np.concatenate((y_test_new, client_data_testy[2]), axis=0)
#test_batched1 = tf.data.Dataset.from_tensor_slices((x_test_new, y_test_new)).batch(len(y_test_new))
x_test = x_test_new
y_test = y_test_new

In [9]:
def k_nearest_neighbors(data, predict, k):
    #k=8
    #if len(data) >= k:
    #    warnings.warn('K is set to a value less than total voting groups!')

    distances = []
    count = 0
    for sample in data:
        euclidean_distance = np.linalg.norm(np.array(sample)-np.array(predict))
        distances.append([euclidean_distance,count])
        count+=1
    
    votes = [i[1] for i in sorted(distances)[:k]] ##votes is returning indexes of k random samples

    #vote_result = Counter(votes).most_common(9)[0][0]
    return votes
def fair_kSMOTE_algo_2(dmajor,dminor,k,r):
    S = []
    Ns =  int(r*(len(dmajor)))
    #Ns = 12
    Nks = int(Ns / (k-1))
    difference = Ns-Nks*(k-1)
    
    rb = []
    #pick a random k sample from dmin and save them in rb
    dmin_rand = random.sample(dminor, k-1)   
    #for debugging
    sens_attr_vals = []
    i = 0
    
    #do algorithem (choose the nearest neighbor and linear interpolation)
    for xb in dmin_rand:
        N= k_nearest_neighbors(dminor,xb,k) #from minority-p
        
        #do linear interpolation
        Sxb = []
        
        if i==0:
            Nkss = Nks+difference
        else:
            Nkss = Nks
        
        i+=1
        for s in range(Nkss):
            
            
            j = 1  ##?? j?
            #randome k sample
            #j = random.randint(0, len(N))
            
            ##here nearst neghber insted of dminor
            #linear interpolation
            x_new = ((dminor[N[j]]-xb) * random.sample(range(0, 1), 1))
            j+=1
            
            while(j < len(N)):
                #here on random xb
                ind = N[j]
                x_new = x_new + ((dminor[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
                
            x_new = x_new / (len(N)-1) ##??? why scaling with k here
            Synthesized_instance = xb + x_new ##?> why we need to sum xb and x_new
            
            
            #for algo 3 when finding nearest neighbors from min_np and assigning the 
            #'p' sensitive value to all synthesized instances
            Synthesized_instance[sa_index] = xb[sa_index] 
            Sxb.append(Synthesized_instance)
        
            
            
            
            
        S.append(Sxb)
    
    
    
    return S

def fair_kSMOTE(dmajor,dminor_wg,dminor,k,r):
    S = []
    #Ns =  int(r*(len(dmajor) - len(dminor)))
    Ns =  int(r*(len(dmajor)))
    #Ns = 12
    
    Nks = int(Ns / (k-1))
    difference = Ns-Nks*(k-1)
    #if r==-1:
    #    Nks = 1
    rb = []
    #pick a random k sample from dmin and save them in rb
    dmin_rand = random.sample(dminor, k-1)   
    #for debugging
    sens_attr_vals = []
    
    
    #do algorithem (choose the nearest neighbor and linear interpolation)
    i = 0
    
    for xb in dmin_rand:
        N= k_nearest_neighbors(dminor,xb,int(k/2)+1) #from minority-p
        N2= k_nearest_neighbors(dminor_wg,xb,int(k/2)) #from minority-np
    
        N3 = np.hstack((N, N2))
        if i==0:
            Nkss = Nks+difference
        else:
            Nkss = Nks
        
        i+=1
        #do linear interpolation
        Sxb = []
        
        for s in range(Nkss):
            
            j = 1  ##?? j?
            #randome k sample
            #j = random.randint(0, len(N))
            
            ##here nearst neghber insted of dminor
            #linear interpolation
            x_new = ((dminor[N[j]]-xb) * random.sample(range(0, 1), 1))
            j+=1
            
            while(j < len(N)):
                #here on random xb
                ind = N[j]
                
                x_new = x_new + ((dminor[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
            j = 0
            while(j < len(N2)):
                #here on random xb
                ind = N2[j]
                
                x_new = x_new + ((dminor_wg[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1    
            x_new = x_new / (len(N3)-1) ##??? why scaling with k here
            Synthesized_instance = xb + x_new ##?> why we need to sum xb and x_new
            
            
            #for algo 3 when finding nearest neighbors from min_np and assigning the 
            #'p' sensitive value to all synthesized instances
            Synthesized_instance[sa_index] = xb[sa_index] 
            Sxb.append(Synthesized_instance)
        
            
            
            
            
        S.append(Sxb)
    
    
   
    return S





def splitYtrain_sa_value(Xtr,Ytr,minority_lable,majority_label,pp_Group=p_Group,npp_Group=np_Group): #splite Ytrain based on sensitive attribute value
    #print(Ytr)
    dmaj_p_x = []
    dmaj_np_x = []
    dmin_p_x = []
    dmin_np_x = []
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==pp_Group): #select minority instances with "protected" value 
            dmin_p_x.append(Xtr[i])
        elif((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==npp_Group): #select minority instances with "protected" value 
            dmin_np_x.append(Xtr[i])
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==pp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_p_x.append(Xtr[i])
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==npp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_np_x.append(Xtr[i])
    
    return dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x



#create_synth_data(window, window_label, minority_label,majority_label,5,lambda_score, 'min_p')
def create_synth_data(x, y, minority_lable,majority_label,k,r,group,pp_group,npp_group):
    
    
    dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x = splitYtrain_sa_value(x,y,minority_lable,majority_label, pp_group,npp_group)
    '''
    print("length of dmin_p_x: " + str(len(dmin_p_x)))
    print("length of dmin_np_x: " + str(len(dmin_np_x)))
    print("length of dmaj_p_x: " + str(len(dmaj_p_x)))
    print("length of dmaj_np_x: " + str(len(dmaj_np_x)))
    '''
    if len(dmin_p_x)<4:
        return -1, -1
    group_names = ['dmin_p_x', 'dmin_np_x', 'dmaj_p_x', 'dmaj_np_x']
    
    Xtr_new = []
    Ytr_new = []  
    
    
    ##Algo 3:
    
    if group =='min_p':
        dmaj_x = dmaj_p_x
        dmin_x = dmin_p_x
        
        #dmin_np = dmin_np_x
        x_syn = fair_kSMOTE(dmaj_x,dmin_np_x,dmin_x,k,r)
        #x_syn = fair_kSMOTE_algo_2(dmaj_x,dmin_x,k,r)
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        new_label = minority_label
        for j in x_syn:
            for s in j:
                Xtr_new.append(s)
                Ytr_new.append(new_label)
        
    elif group =='maj_np':
        dmaj_x = dmin_np_x
        dmin_x = dmaj_np_x
        #dmin_np = dmin_np_x
        x_syn = fair_kSMOTE_algo_2(dmaj_x,dmin_x,k,r)
        
        
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        new_label = majority_label
        for j in x_syn:
            for s in j:
                Xtr_new.append(s)
                Ytr_new.append(new_label)
    
    
    
    return Xtr_new,Ytr_new

In [10]:
def online_update_classSize(train_data, label,classSize):
    theta = 0.9
    if (label not in classSize):
        up_dict = {train_data[-1]:0.5}
        classSize.update(up_dict)
    for classValue in classSize:
        if classValue == label:
            update = theta * classSize.get(classValue) + (1-theta)
            classSize[classValue] = update
        else:
            update = theta * classSize.get(classValue)
            classSize[classValue] = update

In [11]:
def create_clients(Xtr1, Ytr1,Xtr2,Ytr2,Xtr3,Ytr3,num_clients,initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''
    clients = {}
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    
    data = list(zip(Xtr1, Ytr1))
    random.shuffle(data)
    
    clients.update({client_names[0] :data})
    data = list(zip(Xtr2, Ytr2))
    clients.update({client_names[1] :data})
    data = list(zip(Xtr3, Ytr3))
    clients.update({client_names[2] :data})
    #data = list(zip(Xtr4, Ytr4))
    #clients.update({client_names[3] :data})

    return clients
clients = create_clients(Xtr1,Ytr1,Xtr2,Ytr2,Xtr3,Ytr3, num_clients=3, initial='client')

In [12]:
from skmultiflow.drift_detection.eddm import EDDM
from sklearn.utils import shuffle

client_index = {}
client_window = {}
client_window_label = {}
client_eddm = {}

for (client_name, data) in clients.items():
    data, label = zip(*data)
    Y = np.asarray(label)
    X = np.asarray(data)
    client_index.update({client_name:0})
    client_window.update({client_name:[]})
    client_window_label.update({client_name:[]})
    length = len(data)
    print(len(data))
#client_eddm.update({'client_1':eddm1})
#client_eddm.update({'client_2':eddm2})
#client_eddm.update({'client_3':eddm3})
print(client_index)

7694
8990
7315
{'client_1': 0, 'client_2': 0, 'client_3': 0}


In [13]:
for (client_name, data) in clients.items():
    print(client_name)
    data, label = zip(*data)
    Y = np.asarray(label)
    X = np.asarray(data)
    print(np.unique(Y))

client_1
[0 1]
client_2
[0 1]
client_3
[0 1]


In [14]:
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)
from skmultiflow.drift_detection.eddm import EDDM
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight

num_clients = 3
#adwin = ADWIN(delta =1)
window=[]
window_label = []
window_warning = []
window_label_warning = []
pos_assigned=0
pos_samples = 0
neg_samples = 0
pos_syn_samples = 0
neg_syn_samples = 0
generated_samples_per_sample = 0
imbalance_ratio = 0 #of window
minority_label=1
majority_label = 0
lambda_initial=0.001
###
ocis = 0
classSize  = {}
class_weights_dict = {}
labels = []
###
j =0
change=0
warning=0


bal_acc_global=[] 
stp_score_global=[]
gmean_global = []
eddm1 = EDDM()
eddm2 = EDDM()
eddm3 = EDDM()

#one global network
global_network = ONN(features_size=170, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)

#3 networks for three clients
onn_network_1 = ONN(features_size=170, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)
onn_network_2 = ONN(features_size=170, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)
onn_network_3 = ONN(features_size=170, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)
weight = 1
client_down =0 
for _ in range(length):
    if client_down==2:
        break
    sum_w_output_layer, sum_b_output_layer, sum_w_hidden_layer, sum_b_hidden_layer = [],[],[],[]
    sum_alpha = []

    for (client_name, data) in clients.items():
        if client_name not in client_index:
            continue
        added_points = 0
        data, label = zip(*data)
        Y = np.asarray(label)
        X = np.asarray(data)
        
        if client_name=='client_1':
            eddm = eddm1
            onn_network = onn_network_1
        elif client_name=='client_2':
            eddm = eddm2
            onn_network = onn_network_2
        else:
            eddm = eddm3
            onn_network = onn_network_3
        i = client_index[client_name]
        if i ==0:
            print(client_index)
        else:
            if i%200==0:
                galpha, gw_output_layer, gb_output_layer, gw_hidden_layer, gb_hidden_layer = global_network.get_weights('global')
                onn_network.set_weights(galpha, gw_output_layer, gb_output_layer, gw_hidden_layer, gb_hidden_layer)
        
        if np.size(client_window[client_name])!=0:
            if client_window[client_name].ndim==2 and len(client_window[client_name])>30:
                majority_count = list(client_window_label[client_name]).count(0)
                minority_count = list(client_window_label[client_name]).count(1)
                if majority_count >  minority_count and minority_count!=0:
                    weight = int(majority_count/minority_count)
        
        if client_index[client_name]<len(Y):
            if Y[i]==minority_label:
                onn_network.partial_fit(np.asarray([X[i, :]]), np.asarray([Y[i]]),2)
            
            else:
                onn_network.partial_fit(np.asarray([X[i, :]]), np.asarray([Y[i]]),1)
        
        
        
        #onn_network.partial_fit(np.asarray([X[i, :]]), np.asarray([Y[i]]))
    
            if np.size(client_window[client_name])==0:
                client_window[client_name] = np.array(X[i])
                client_window_label[client_name] = np.array(Y[i])
            else:
                client_window[client_name]=np.vstack((client_window[client_name],np.array(X[i])))
                client_window_label[client_name]= np.vstack((client_window_label[client_name],np.array(Y[i])))
            eddm.add_element(Y[i])
    

            if eddm.detected_change():
                print('Change has been detected in data: ' + str(Y[i]) + ' - of index: ' + str(i))
                change+=1
                client_window[client_name] = []
                client_window_label[client_name] = []
    
            
            pos_assigned = onn_network.tp+onn_network.fp-0.2
            pos_samples = onn_network.tp+onn_network.fn-0.2
            
            if np.size(client_window[client_name])!=0:
                if client_window[client_name].ndim==2 and len(client_window[client_name])>30:
                    if onn_network.eqop_score > 0.005:
                        #print(onn_network.stp_score)
                        lambda_score = lambda_initial*(1+(onn_network.stp_score/0.2))
                        if pos_assigned <= pos_samples:
                            X_syn,Y_syn = create_synth_data(client_window[client_name], client_window_label[client_name], minority_label,majority_label,4,lambda_score, 'min_p', p_Group,np_Group)
                        else:
                            X_syn,Y_syn = create_synth_data(client_window[client_name], client_window_label[client_name], minority_label,majority_label,4,lambda_score, 'maj_np', p_Group,np_Group)
                        if X_syn!=-1:
                            Y_syn = np.array(Y_syn)
                            X_syn = np.array(X_syn)
                            X_syn, Y_syn = shuffle(X_syn, Y_syn, random_state=0)
                            added_points = len(Y_syn)
                            for k in range(len(X_syn)):
                                onn_network.partial_fit(np.asarray([X_syn[k, :]]), np.asarray([Y_syn[k]]), 1, test='no')
            
            client_alpha, client_w_output_layer, client_b_output_layer, client_w_hidden_layer, client_b_hidden_layer = onn_network.get_weights('client')
            i=i+1
            client_index.update({client_name:i})
            scaling_factor = (client_index[client_name]+added_points)/(sum(client_index.values())+added_points)
            #scaling_factor = 1/5
            scaling_factor2 = 1/3
            print(client_index)
            p = 0
            if p==0:
                if sum_alpha==[]:
                    sum_alpha = torch.mul(client_alpha, scaling_factor2)
                    sum_w_output_layer = client_w_output_layer
                    sum_b_output_layer = client_b_output_layer
                    sum_w_hidden_layer = client_w_hidden_layer
                    sum_b_hidden_layer = client_b_hidden_layer
            
            
            
            
                    for j in range(onn_network.max_num_hidden_layers):
                        sum_w_output_layer[j] = torch.mul(client_w_output_layer[j], scaling_factor)
                        sum_b_output_layer[j] = torch.mul(client_b_output_layer[j], scaling_factor)
                        sum_w_hidden_layer[j] = torch.mul(client_w_hidden_layer[j], scaling_factor)
                        sum_b_hidden_layer[j] = torch.mul(client_b_hidden_layer[j], scaling_factor)

                else:
                    sum_alpha = torch.add(sum_alpha, torch.mul(client_alpha, scaling_factor))
                    for j in range(onn_network.max_num_hidden_layers):
                        sum_w_output_layer[j] = torch.add(sum_w_output_layer[j],torch.mul(client_w_output_layer[j], scaling_factor)) 
                        sum_b_output_layer[j] = torch.add(sum_b_output_layer[j],torch.mul(client_b_output_layer[j], scaling_factor))
                        sum_w_hidden_layer[j] = torch.add(sum_w_hidden_layer[j],torch.mul(client_w_hidden_layer[j], scaling_factor)) 
                        sum_b_hidden_layer[j] = torch.add(sum_b_hidden_layer[j],torch.mul(client_b_hidden_layer[j], scaling_factor))
        if len(Y)==client_index[client_name]:
            client_down+=1
            del client_index[client_name]
    if i%200==0:    
        global_network.set_weights(sum_alpha, sum_w_output_layer, sum_b_output_layer, sum_w_hidden_layer, sum_b_hidden_layer)

        x_test, y_test = shuffle(x_test, y_test, random_state=0)
        
        for m in range(len(x_test)-1):
            prediction_1 = global_network.predict_1(np.asarray([x_test[m, :]]))
            global_network.update_eval_metrics(prediction_1,np.asarray([y_test[m]]))
            global_network.update_stp_score(prediction_1,np.asarray([x_test[m, :]]))
            global_network.update_eqop_score(prediction_1,np.asarray([x_test[m, :]]),np.asarray([y_test[m]]))
        
        print("Balanced accuracy: " + str(global_network.bal_acc))
        print("Sensitivity: " + str(global_network.sen))
        print("Specificity: " + str(global_network.spec))
        print("Stp score: " + str(global_network.stp_score))
        print("Eqop score: " + str(global_network.eqop_score))
        
        bal_acc_global.append(global_network.bal_acc)
        gmean_global.append(math.sqrt(global_network.sen*global_network.spec))
        stp_score_global.append(global_network.stp_score)
        global_network.reset_eval_metrics()
global_network.set_weights(sum_alpha, sum_w_output_layer, sum_b_output_layer, sum_w_hidden_layer, sum_b_hidden_layer) 
for n in range(len(x_test)-1):
    prediction_1 = global_network.predict_1(np.asarray([x_test[n, :]]))
    global_network.update_eval_metrics(prediction_1,np.asarray([y_test[n]]))
    global_network.update_stp_score(prediction_1,np.asarray([x_test[n, :]]))
    global_network.update_eqop_score(prediction_1,np.asarray([x_test[n, :]]),np.asarray([y_test[n]]))

bal_acc_global.append(global_network.bal_acc)  
stp_score_global.append(global_network.stp_score)        #print("change" + str(change))
gmean_global.append(math.sqrt(global_network.sen*global_network.spec))
#print("warning" + str(warning))
    
    
    
    
    
print("Balanced accuracy: " + str(onn_network.bal_acc))
print("Sensitivity: " + str(onn_network.sen))
print("Specificity: " + str(onn_network.spec))
print("Stp score: " + str(onn_network.stp_score))
print("Eqop score: " + str(onn_network.eqop_score))

{'client_1': 0, 'client_2': 0, 'client_3': 0}
{'client_1': 1, 'client_2': 0, 'client_3': 0}
{'client_1': 1, 'client_2': 0, 'client_3': 0}
{'client_1': 1, 'client_2': 1, 'client_3': 0}
{'client_1': 1, 'client_2': 1, 'client_3': 0}
{'client_1': 1, 'client_2': 1, 'client_3': 1}
{'client_1': 2, 'client_2': 1, 'client_3': 1}
{'client_1': 2, 'client_2': 2, 'client_3': 1}
{'client_1': 2, 'client_2': 2, 'client_3': 2}
{'client_1': 3, 'client_2': 2, 'client_3': 2}
{'client_1': 3, 'client_2': 3, 'client_3': 2}
{'client_1': 3, 'client_2': 3, 'client_3': 3}
{'client_1': 4, 'client_2': 3, 'client_3': 3}
{'client_1': 4, 'client_2': 4, 'client_3': 3}
{'client_1': 4, 'client_2': 4, 'client_3': 4}
{'client_1': 5, 'client_2': 4, 'client_3': 4}
{'client_1': 5, 'client_2': 5, 'client_3': 4}
{'client_1': 5, 'client_2': 5, 'client_3': 5}
{'client_1': 6, 'client_2': 5, 'client_3': 5}
{'client_1': 6, 'client_2': 6, 'client_3': 5}
{'client_1': 6, 'client_2': 6, 'client_3': 6}
{'client_1': 7, 'client_2': 6, 'cl

{'client_1': 56, 'client_2': 56, 'client_3': 56}
{'client_1': 57, 'client_2': 56, 'client_3': 56}
{'client_1': 57, 'client_2': 57, 'client_3': 56}
{'client_1': 57, 'client_2': 57, 'client_3': 57}
{'client_1': 58, 'client_2': 57, 'client_3': 57}
{'client_1': 58, 'client_2': 58, 'client_3': 57}
{'client_1': 58, 'client_2': 58, 'client_3': 58}
{'client_1': 59, 'client_2': 58, 'client_3': 58}
{'client_1': 59, 'client_2': 59, 'client_3': 58}
{'client_1': 59, 'client_2': 59, 'client_3': 59}
{'client_1': 60, 'client_2': 59, 'client_3': 59}
{'client_1': 60, 'client_2': 60, 'client_3': 59}
{'client_1': 60, 'client_2': 60, 'client_3': 60}
{'client_1': 61, 'client_2': 60, 'client_3': 60}
{'client_1': 61, 'client_2': 61, 'client_3': 60}
{'client_1': 61, 'client_2': 61, 'client_3': 61}
{'client_1': 62, 'client_2': 61, 'client_3': 61}
{'client_1': 62, 'client_2': 62, 'client_3': 61}
{'client_1': 62, 'client_2': 62, 'client_3': 62}
{'client_1': 63, 'client_2': 62, 'client_3': 62}
{'client_1': 63, 'cl

{'client_1': 113, 'client_2': 112, 'client_3': 112}
{'client_1': 113, 'client_2': 113, 'client_3': 112}
{'client_1': 113, 'client_2': 113, 'client_3': 113}
{'client_1': 114, 'client_2': 113, 'client_3': 113}
{'client_1': 114, 'client_2': 114, 'client_3': 113}
{'client_1': 114, 'client_2': 114, 'client_3': 114}
{'client_1': 115, 'client_2': 114, 'client_3': 114}
{'client_1': 115, 'client_2': 115, 'client_3': 114}
{'client_1': 115, 'client_2': 115, 'client_3': 115}
{'client_1': 116, 'client_2': 115, 'client_3': 115}
{'client_1': 116, 'client_2': 116, 'client_3': 115}
{'client_1': 116, 'client_2': 116, 'client_3': 116}
{'client_1': 117, 'client_2': 116, 'client_3': 116}
{'client_1': 117, 'client_2': 117, 'client_3': 116}
{'client_1': 117, 'client_2': 117, 'client_3': 117}
{'client_1': 118, 'client_2': 117, 'client_3': 117}
{'client_1': 118, 'client_2': 118, 'client_3': 117}
{'client_1': 118, 'client_2': 118, 'client_3': 118}
{'client_1': 119, 'client_2': 118, 'client_3': 118}
{'client_1':

{'client_1': 166, 'client_2': 165, 'client_3': 165}
{'client_1': 166, 'client_2': 166, 'client_3': 165}
{'client_1': 166, 'client_2': 166, 'client_3': 166}
{'client_1': 167, 'client_2': 166, 'client_3': 166}
{'client_1': 167, 'client_2': 167, 'client_3': 166}
{'client_1': 167, 'client_2': 167, 'client_3': 167}
{'client_1': 168, 'client_2': 167, 'client_3': 167}
{'client_1': 168, 'client_2': 168, 'client_3': 167}
{'client_1': 168, 'client_2': 168, 'client_3': 168}
{'client_1': 169, 'client_2': 168, 'client_3': 168}
{'client_1': 169, 'client_2': 169, 'client_3': 168}
{'client_1': 169, 'client_2': 169, 'client_3': 169}
{'client_1': 170, 'client_2': 169, 'client_3': 169}
{'client_1': 170, 'client_2': 170, 'client_3': 169}
{'client_1': 170, 'client_2': 170, 'client_3': 170}
{'client_1': 171, 'client_2': 170, 'client_3': 170}
{'client_1': 171, 'client_2': 171, 'client_3': 170}
{'client_1': 171, 'client_2': 171, 'client_3': 171}
{'client_1': 172, 'client_2': 171, 'client_3': 171}
{'client_1':

{'client_1': 218, 'client_2': 218, 'client_3': 218}
{'client_1': 219, 'client_2': 218, 'client_3': 218}
{'client_1': 219, 'client_2': 219, 'client_3': 218}
{'client_1': 219, 'client_2': 219, 'client_3': 219}
{'client_1': 220, 'client_2': 219, 'client_3': 219}
{'client_1': 220, 'client_2': 220, 'client_3': 219}
{'client_1': 220, 'client_2': 220, 'client_3': 220}
{'client_1': 221, 'client_2': 220, 'client_3': 220}
{'client_1': 221, 'client_2': 221, 'client_3': 220}
{'client_1': 221, 'client_2': 221, 'client_3': 221}
{'client_1': 222, 'client_2': 221, 'client_3': 221}
{'client_1': 222, 'client_2': 222, 'client_3': 221}
{'client_1': 222, 'client_2': 222, 'client_3': 222}
{'client_1': 223, 'client_2': 222, 'client_3': 222}
{'client_1': 223, 'client_2': 223, 'client_3': 222}
{'client_1': 223, 'client_2': 223, 'client_3': 223}
{'client_1': 224, 'client_2': 223, 'client_3': 223}
{'client_1': 224, 'client_2': 224, 'client_3': 223}
{'client_1': 224, 'client_2': 224, 'client_3': 224}
{'client_1':

...

{'client_1': 7075, 'client_2': 7074, 'client_3': 7074}
{'client_1': 7075, 'client_2': 7075, 'client_3': 7074}
{'client_1': 7075, 'client_2': 7075, 'client_3': 7075}
{'client_1': 7076, 'client_2': 7075, 'client_3': 7075}
{'client_1': 7076, 'client_2': 7076, 'client_3': 7075}
{'client_1': 7076, 'client_2': 7076, 'client_3': 7076}
{'client_1': 7077, 'client_2': 7076, 'client_3': 7076}
{'client_1': 7077, 'client_2': 7077, 'client_3': 7076}
{'client_1': 7077, 'client_2': 7077, 'client_3': 7077}
{'client_1': 7078, 'client_2': 7077, 'client_3': 7077}
{'client_1': 7078, 'client_2': 7078, 'client_3': 7077}
{'client_1': 7078, 'client_2': 7078, 'client_3': 7078}
{'client_1': 7079, 'client_2': 7078, 'client_3': 7078}
{'client_1': 7079, 'client_2': 7079, 'client_3': 7078}
{'client_1': 7079, 'client_2': 7079, 'client_3': 7079}
{'client_1': 7080, 'client_2': 7079, 'client_3': 7079}
{'client_1': 7080, 'client_2': 7080, 'client_3': 7079}
{'client_1': 7080, 'client_2': 7080, 'client_3': 7080}
{'client_1

{'client_1': 7125, 'client_2': 7125, 'client_3': 7124}
{'client_1': 7125, 'client_2': 7125, 'client_3': 7125}
{'client_1': 7126, 'client_2': 7125, 'client_3': 7125}
{'client_1': 7126, 'client_2': 7126, 'client_3': 7125}
{'client_1': 7126, 'client_2': 7126, 'client_3': 7126}
{'client_1': 7127, 'client_2': 7126, 'client_3': 7126}
{'client_1': 7127, 'client_2': 7127, 'client_3': 7126}
{'client_1': 7127, 'client_2': 7127, 'client_3': 7127}
{'client_1': 7128, 'client_2': 7127, 'client_3': 7127}
{'client_1': 7128, 'client_2': 7128, 'client_3': 7127}
{'client_1': 7128, 'client_2': 7128, 'client_3': 7128}
{'client_1': 7129, 'client_2': 7128, 'client_3': 7128}
{'client_1': 7129, 'client_2': 7129, 'client_3': 7128}
{'client_1': 7129, 'client_2': 7129, 'client_3': 7129}
{'client_1': 7130, 'client_2': 7129, 'client_3': 7129}
{'client_1': 7130, 'client_2': 7130, 'client_3': 7129}
{'client_1': 7130, 'client_2': 7130, 'client_3': 7130}
{'client_1': 7131, 'client_2': 7130, 'client_3': 7130}
{'client_1

{'client_1': 7174, 'client_2': 7174, 'client_3': 7173}
{'client_1': 7174, 'client_2': 7174, 'client_3': 7174}
{'client_1': 7175, 'client_2': 7174, 'client_3': 7174}
{'client_1': 7175, 'client_2': 7175, 'client_3': 7174}
{'client_1': 7175, 'client_2': 7175, 'client_3': 7175}
{'client_1': 7176, 'client_2': 7175, 'client_3': 7175}
{'client_1': 7176, 'client_2': 7176, 'client_3': 7175}
{'client_1': 7176, 'client_2': 7176, 'client_3': 7176}
{'client_1': 7177, 'client_2': 7176, 'client_3': 7176}
{'client_1': 7177, 'client_2': 7177, 'client_3': 7176}
{'client_1': 7177, 'client_2': 7177, 'client_3': 7177}
{'client_1': 7178, 'client_2': 7177, 'client_3': 7177}
{'client_1': 7178, 'client_2': 7178, 'client_3': 7177}
{'client_1': 7178, 'client_2': 7178, 'client_3': 7178}
{'client_1': 7179, 'client_2': 7178, 'client_3': 7178}
{'client_1': 7179, 'client_2': 7179, 'client_3': 7178}
{'client_1': 7179, 'client_2': 7179, 'client_3': 7179}
{'client_1': 7180, 'client_2': 7179, 'client_3': 7179}
{'client_1

{'client_1': 7224, 'client_2': 7223, 'client_3': 7223}
{'client_1': 7224, 'client_2': 7224, 'client_3': 7223}
{'client_1': 7224, 'client_2': 7224, 'client_3': 7224}
{'client_1': 7225, 'client_2': 7224, 'client_3': 7224}
{'client_1': 7225, 'client_2': 7225, 'client_3': 7224}
{'client_1': 7225, 'client_2': 7225, 'client_3': 7225}
{'client_1': 7226, 'client_2': 7225, 'client_3': 7225}
{'client_1': 7226, 'client_2': 7226, 'client_3': 7225}
{'client_1': 7226, 'client_2': 7226, 'client_3': 7226}
{'client_1': 7227, 'client_2': 7226, 'client_3': 7226}
{'client_1': 7227, 'client_2': 7227, 'client_3': 7226}
{'client_1': 7227, 'client_2': 7227, 'client_3': 7227}
{'client_1': 7228, 'client_2': 7227, 'client_3': 7227}
{'client_1': 7228, 'client_2': 7228, 'client_3': 7227}
{'client_1': 7228, 'client_2': 7228, 'client_3': 7228}
{'client_1': 7229, 'client_2': 7228, 'client_3': 7228}
{'client_1': 7229, 'client_2': 7229, 'client_3': 7228}
{'client_1': 7229, 'client_2': 7229, 'client_3': 7229}
{'client_1

{'client_1': 7274, 'client_2': 7274, 'client_3': 7273}
{'client_1': 7274, 'client_2': 7274, 'client_3': 7274}
{'client_1': 7275, 'client_2': 7274, 'client_3': 7274}
{'client_1': 7275, 'client_2': 7275, 'client_3': 7274}
{'client_1': 7275, 'client_2': 7275, 'client_3': 7275}
{'client_1': 7276, 'client_2': 7275, 'client_3': 7275}
{'client_1': 7276, 'client_2': 7276, 'client_3': 7275}
{'client_1': 7276, 'client_2': 7276, 'client_3': 7276}
{'client_1': 7277, 'client_2': 7276, 'client_3': 7276}
{'client_1': 7277, 'client_2': 7277, 'client_3': 7276}
{'client_1': 7277, 'client_2': 7277, 'client_3': 7277}
{'client_1': 7278, 'client_2': 7277, 'client_3': 7277}
{'client_1': 7278, 'client_2': 7278, 'client_3': 7277}
{'client_1': 7278, 'client_2': 7278, 'client_3': 7278}
{'client_1': 7279, 'client_2': 7278, 'client_3': 7278}
{'client_1': 7279, 'client_2': 7279, 'client_3': 7278}
{'client_1': 7279, 'client_2': 7279, 'client_3': 7279}
{'client_1': 7280, 'client_2': 7279, 'client_3': 7279}
{'client_1

In [17]:
print(stp_score_global)

[-0.00016641027502961592, 0.0002588037802520038, 0.0002589044578727995, 0.0002589044578727995, -0.00016641027502961592, -0.00016669192224139096, -4.427244200500002e-05, 0.0002938673678115622, -0.0007258908658612784, 0.001723104891309335, 0.010867204013113807, 0.08074845801462482, 0.037115831658647847, 0.05497574041686609, 0.04457731337112941, 0.13441623959613352, 0.17354874200730308, 0.1329901081201053, 0.05574935355703731, 0.08788087230368735, 0.06895529593549005, 0.08599223618500595, 0.0652874432023246, 0.09811896202729647, 0.1340783133700677, 0.07933942612655587, 0.11515943013438851, 0.03067329534464061, -0.006345944765979827, -0.04921531211388527, -0.03534684725962281, -0.05089290894474496, -0.1141842698115832, -0.2984448585672569, -0.22108657385810632, -0.10990019415575691, -0.23800836845672016]


In [15]:
import math
onn_network = global_network
print("Balanced accuracy: " + str(onn_network.bal_acc))
print("Sensitivity: " + str(onn_network.sen))
print("Specificity: " + str(onn_network.spec))
print("Gmean: " + str(math.sqrt(onn_network.sen*onn_network.spec)))
print("Stp score: " + str(onn_network.stp_score))
print("Eqop score: " + str(onn_network.eqop_score))

Balanced accuracy: 0.6885060993368063
Sensitivity: 0.6321552635553541
Specificity: 0.7448569351182585
Gmean: 0.6861962052727456
Stp score: -0.18051233785765983
Eqop score: -0.22224306137375294


In [16]:
print(len(x_test))
print(len(y_test))
x_test, y_test = shuffle(x_test, y_test, random_state=0)
for i in range(len(x_test)-1):
    prediction_1 = global_network.predict_1(np.asarray([x_test[i, :]]))
    global_network.update_eval_metrics(prediction_1,np.asarray([y_test[i]]))
    global_network.update_stp_score(prediction_1,np.asarray([x_test[i, :]]))
    global_network.update_eqop_score(prediction_1,np.asarray([x_test[i, :]]),np.asarray([y_test[i]]))

6000
6000


In [15]:
print(stp_score_global)

[1.4992821898192745e-05, 1.4967280755520938e-05, 1.4992821898192745e-05, 0.0034102903348427645, 0.00922318783223112, -0.023668046427093192, -0.11485766621511212, 0.07438520723703924, -0.02612556880459692, 0.0004858600639338906, -0.04438221270062705, -0.02619310240053005, -0.0010738121246165189, 0.09993732564748596, -0.04844668838162468, -0.04966429918475956, -0.041010710348495794, 0.0029149623728851465, -0.02950010702568249, 0.006134370744345169, -0.04457811803773931, 0.028337586681576488, 0.03557888032775444, -0.031071317748656707, -0.034938979572776335, -0.025894711895942557, -0.09209880347201407, -0.06378984400084142, -0.021475487428172874, -0.0056753597354620156, 0.04299011866880181, 0.052750513907802915, -0.0013931684699510505, -0.03771695347175827, -0.05610659942501375, 0.018774549310599825, 0.07425022050981131, -0.06444548850189964, -0.05473648616231738, 0.0029789583817716736, 0.0029789583817716736]


In [32]:
print("Balanced accuracy: " + str(onn_network.bal_acc))
print("Sensitivity: " + str(onn_network.sen))
print("Specificity: " + str(onn_network.spec))
print("Gmean: " + str(math.sqrt(onn_network.sen*onn_network.spec)))
print("Stp score: " + str(onn_network.stp_score))
print("Eqop score: " + str(onn_network.eqop_score))

Balanced accuracy: 0.5980407567413166
Sensitivity: 0.8620870172173104
Specificity: 0.3339944962653229
Gmean: 0.5365932528949374
Stp score: -0.016893744372228237
Eqop score: -0.01944045904814795


In [19]:
print("Balanced accuracy: " + str(onn_network.bal_acc))
print("Sensitivity: " + str(onn_network.sen))
print("Specificity: " + str(onn_network.spec))
print("Gmean: " + str(math.sqrt(onn_network.sen*onn_network.spec)))
print("Stp score: " + str(onn_network.stp_score))
print("Eqop score: " + str(onn_network.eqop_score))

Balanced accuracy: 0.529750386996904
Sensitivity: 0.9871323529411764
Specificity: 0.07236842105263158


NameError: name 'math' is not defined

6000
6000


In [14]:
sum_w_output_layer, sum_b_output_layer, sum_w_hidden_layer, sum_b_hidden_layer = [],[],[],[]
sum_alpha = []
scaling_factor = 1/5
global_network2 = ONN(features_size=46, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)
for m in range(3):
    if m==0:
        onn_network=onn_network_1
    elif m==1:
        onn_network=onn_network_2
    else:
        onn_network=onn_network_3
    if sum_alpha==[]:
        sum_alpha = torch.mul(client_alpha, scaling_factor)
        sum_w_output_layer = client_w_output_layer
        sum_b_output_layer = client_b_output_layer
        sum_w_hidden_layer = client_w_hidden_layer
        sum_b_hidden_layer = client_b_hidden_layer
            
        for j in range(onn_network.max_num_hidden_layers):
            sum_w_output_layer[j] = torch.mul(client_w_output_layer[j], scaling_factor)
            sum_b_output_layer[j] = torch.mul(client_b_output_layer[j], scaling_factor)
            sum_w_hidden_layer[j] = torch.mul(client_w_hidden_layer[j], scaling_factor)
            sum_b_hidden_layer[j] = torch.mul(client_b_hidden_layer[j], scaling_factor)

    else:
        sum_alpha = torch.add(sum_alpha, torch.mul(client_alpha, scaling_factor))
        for j in range(onn_network.max_num_hidden_layers):
            sum_w_output_layer[j] = torch.add(sum_w_output_layer[j],torch.mul(client_w_output_layer[j], scaling_factor)) 
            sum_b_output_layer[j] = torch.add(sum_b_output_layer[j],torch.mul(client_b_output_layer[j], scaling_factor))
            sum_w_hidden_layer[j] = torch.add(sum_w_hidden_layer[j],torch.mul(client_w_hidden_layer[j], scaling_factor)) 
            sum_b_hidden_layer[j] = torch.add(sum_b_hidden_layer[j],torch.mul(client_b_hidden_layer[j], scaling_factor))
        
        
global_network2.set_weights(sum_alpha, sum_w_output_layer, sum_b_output_layer, sum_w_hidden_layer, sum_b_hidden_layer)


In [23]:
onn_network_4 = ONN(features_size=46, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=2)
galpha, gw_output_layer, gb_output_layer, gw_hidden_layer, gb_hidden_layer = onn_network_3.get_weights('local')
onn_network_4.set_weights(galpha, gw_output_layer, gb_output_layer, gw_hidden_layer, gb_hidden_layer)